In [35]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import sklearn
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.naive_bayes import GaussianNB 
from sklearn import metrics 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier




data = pd.read_csv('online_shoppers_intention1.csv')
data = pd.read_csv(os.path.join(os.getcwd(),'online_shoppers_intention1.csv'))

In [36]:
data.isnull().sum()
# I have already clean this data 

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

In [37]:
data.columns

Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend', 'Revenue'],
      dtype='object')

In [38]:
data.duplicated().sum()

121

In [39]:
data = data.drop_duplicates()

In [40]:
data.duplicated().sum()

0

In [42]:
data_cleaned = data.drop_duplicates()

# Confirm no duplicates are present
duplicates_count = data_cleaned.duplicated().sum()
duplicates_count

0

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11827 entries, 0 to 11947
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           11827 non-null  int64  
 1   Administrative_Duration  11827 non-null  float64
 2   Informational            11827 non-null  int64  
 3   Informational_Duration   11827 non-null  float64
 4   ProductRelated           11827 non-null  int64  
 5   ProductRelated_Duration  11827 non-null  float64
 6   BounceRates              11827 non-null  float64
 7   ExitRates                11827 non-null  float64
 8   PageValues               11827 non-null  float64
 9   SpecialDay               11827 non-null  float64
 10  Month                    11827 non-null  object 
 11  OperatingSystems         11827 non-null  int64  
 12  Browser                  11827 non-null  int64  
 13  Region                   11827 non-null  int64  
 14  TrafficType              11

In [25]:
encode = {'VisitorType'}
dict_visitor = {'VisitorType' : 'Encode'}
data_enc = data
for i in encode:
  enc = (data_enc.groupby(i).size()) * 100/ len(data)
  dict_visitor[data_enc.groupby(i)] = enc
  enc_name = str(i+'_enc')
  data_enc[enc_name] = data_enc[i].apply(lambda x : enc[x])
  data_enc = data_enc.drop(columns = [i])

In [26]:
dict_visitor

{'VisitorType': 'Encode',
 <pandas.core.groupby.generic.DataFrameGroupBy object at 0x157bc7e50>: VisitorType
 New_Visitor          13.891942
 Other                 0.676418
 Returning_Visitor    85.431639
 dtype: float64}

In [27]:
encode = {'Weekend'}
map_class = {True : 1, False : 0}
for i in encode:
  data_enc[i] = data_enc[i].map(map_class)

In [29]:
map_month = {'Jan' : 1, 'Feb' : 2, 'Mar' : 3, 'Apr' : 4, 'May' : 5, 'June' : 6, 'Jul' : 7, 'Aug' : 8, 'Sep' : 9, 'Oct' : 10, 'Nov' : 11, 'Dec' : 12}
data_enc['Month'] = data_enc['Month'].map(map_month)

In [31]:
data_enc

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,Weekend,Revenue,VisitorType_enc
0,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,2,1,1,1,1,0,False,85.431639
1,0,0.0,0,0.0,2,64.000000,0.000000,0.100000,0.000000,0.0,2,2,2,1,2,0,False,85.431639
2,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,2,4,1,9,3,0,False,85.431639
3,0,0.0,0,0.0,2,2.666667,0.050000,0.140000,0.000000,0.0,2,3,2,2,4,0,False,85.431639
4,0,0.0,0,0.0,10,627.500000,0.020000,0.050000,0.000000,0.0,2,3,3,1,4,1,False,85.431639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11943,3,145.0,0,0.0,53,1783.791667,0.007143,0.029031,12.241717,0.0,12,4,6,1,1,1,False,85.431639
11944,0,0.0,0,0.0,5,465.750000,0.000000,0.021333,0.000000,0.0,11,3,2,1,8,1,False,85.431639
11945,0,0.0,0,0.0,6,184.250000,0.083333,0.086667,0.000000,0.0,11,3,2,1,13,1,False,85.431639
11946,4,75.0,0,0.0,15,346.000000,0.000000,0.021053,0.000000,0.0,11,2,2,3,11,0,False,85.431639
